In [3]:
import pickle
import nest_asyncio

from llama_index.core import (
    KnowledgeGraphIndex,
    StorageContext
)
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.llms.llama_cpp import LlamaCPP

/home/hbjugo/anaconda3/envs/graph_rag/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/hbjugo/anaconda3/envs/graph_rag/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/hbjugo/anaconda3/envs/graph_rag/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [4]:
model_url = "https://huggingface.co/QuantFactory/Meta-Llama-3-8B-Instruct-GGUF/resolve/main/Meta-Llama-3-8B-Instruct.Q4_0.gguf"
llm_llama3 = LlamaCPP(
    model_url=model_url,
    model_path=None,
    temperature=0.1,
    max_new_tokens=512,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    verbose=True,
)

embed_model = HuggingFaceEmbedding(model_name = "dunzhang/stella_en_400M_v5", device = "cuda", trust_remote_code=True, embed_batch_size=20)

total size (MB): 3791.73


  2%|▏         | 75/3616 [00:40<31:53,  1.85it/s] 


Download incomplete. Removing partially downloaded file.


ValueError: Download incomplete.